In [1]:
import numpy as np
from pyscf import gto, scf, fci
from tabulate import tabulate
from RDMFS import HF_2RDM
from utils import R_spectral_clean, R_twoDM_Eigvals
import scipy 

In [2]:
# Get Hydrogen in mo basis from PySCF

mol = gto.Mole()
mol.unit = 'B' 
L = 1.5
mol.atom = f"""
# He 0 0 0 
H   {-L}   {-L}   0.0
H    {L}   {-L}   0.0
H    {L}    {L}   0.0
H   {-L}    {L}   0.0
H   {-L}   {-L}   {L}
H    {L}   {-L}   {L}
H    {L}    {L}   {L}
H   {-L}    {L}   {L}
"""
# this basis has 2 functions for Helium
# mol.basis = "ccpvdz"
# mol.basis = "sto-3g"
mol.basis = "631g"

mol.spin =  0
mol.verbose= 0
mol.build()

# the 2 electron integrals \langle \mu \nu | \kappa \lambda \rangle have M^4 entries
eri_ao = mol.intor('int2e')
S = mol.intor('int1e_ovlp')

## Run Hartree-Fock.
mf = scf.RHF(mol)
mf.kernel()


# Harvest HF quantities

h1 = mf.get_hcore()
M = h1.shape[0]
C_a = mf.mo_coeff
h1_a = C_a.T@h1@C_a
eri_h2_aa = eri_ao.copy()

print(mol.nelec)

for i in range(4):
  eri_h2_aa = np.tensordot(eri_h2_aa, C_a, axes=1).transpose(3, 0, 1, 2)


h1_mo = (h1_a )
h2_mo = (eri_h2_aa )

(4, 4)


In [3]:
dm1_ao = mf.make_rdm1()
dm2_ao = mf.make_rdm2()

# # dm1_mo=C[:,0:N]@dm1_ao@C[:,0:N].T
dm1_mo=(C_a.T@(S.T@dm1_ao@S))@C_a

dm2_mo = dm2_ao.copy()
for i in range(4):
  dm2_mo = np.tensordot(dm2_mo, S@C_a, axes=1).transpose(3, 0, 1, 2)


# P = C@C.T
# print(P)

occ, C = R_spectral_clean(dm1_mo, h1.shape[0])
print(occ, np.sum(occ))
print(dm1_mo.shape, dm1_ao.shape, C_a.shape)

[2.00000000e+00 2.00000000e+00 2.00000000e+00 2.00000000e+00
 3.90638279e-15 1.82585354e-15 9.14352245e-16 5.91700955e-16
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00] 8.000000000000002
(16, 16) (16, 16) (16, 16)


In [4]:
dm2_HF_mo = HF_2RDM(occ, M)

In [5]:
N=2
for i in range(0,N):
    for j in range(0,N):
        for k in range(0,N):
            for l in range(0,N):
                print(i,j,k,l, dm2_HF_mo[i,j,k,l], dm2_mo[i,j,k,l])

0 0 0 0 2.0000000000000036 1.9999999999999956
0 0 0 1 0.0 -2.664085977324235e-16
0 0 1 0 0.0 -3.2891158908618024e-16
0 0 1 1 4.0000000000000036 3.999999999999995
0 1 0 0 0.0 -1.3331661931024693e-16
0 1 0 1 0.0 -3.6268839862535565e-17
0 1 1 0 -2.0000000000000018 -1.9999999999999973
0 1 1 1 0.0 -3.8602474495498034e-16
1 0 0 0 0.0 -4.1065120617458737e-16
1 0 0 1 -2.0000000000000018 -1.9999999999999978
1 0 1 0 0.0 3.027752177006714e-17
1 0 1 1 0.0 -2.8557948970127705e-16
1 1 0 0 4.0000000000000036 3.999999999999996
1 1 0 1 0.0 -2.617780012749776e-16
1 1 1 0 0.0 -2.083096481300116e-16
1 1 1 1 2.0 2.0


In [6]:
print(np.sum(R_twoDM_Eigvals(np.transpose(dm2_HF_mo, (0, 2, 1, 3)))), scipy.special.binom(np.sum(mol.nelec),2))
print(np.sum(R_twoDM_Eigvals(np.transpose(dm2_mo, (0, 2, 1, 3)))), scipy.special.binom(np.sum(mol.nelec),2))

# print(np.sum(twoDM_Eigvals(dm2_HF_mo)), scipy.special.binom(4,2))
# print(np.sum(twoDM_Eigvals(dm2_mo)), scipy.special.binom(4,2))

56.00000000000002 28.0
55.99999999999989 28.0


In [7]:
# Run FCI  
cisolver = fci.direct_spin0.FCI()
cisolver.max_cycle = 100
cisolver.conv_tol = 1e-8
e, ci_mo = cisolver.kernel(h1_mo, h2_mo, h1.shape[0], mol.nelec, ecore=mf.energy_nuc())
dm1_fci, dm2_fci  = cisolver.make_rdm12(ci_mo, h1.shape[0], mol.nelec)
occ, C = R_spectral_clean(dm1_mo, h1.shape[0])
print(occ, np.sum(occ))


[2.00000000e+00 2.00000000e+00 2.00000000e+00 2.00000000e+00
 3.90638279e-15 1.82585354e-15 9.14352245e-16 5.91700955e-16
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00] 8.000000000000002


In [8]:
dm2_HF_fci = HF_2RDM(occ, M)

In [9]:
print(np.sum(R_twoDM_Eigvals(np.transpose(dm2_HF_fci, (0, 2, 1, 3)))), scipy.special.binom(np.sum(mol.nelec),2))
print(np.sum(R_twoDM_Eigvals(np.transpose(dm2_fci, (0, 2, 1, 3)))), scipy.special.binom(np.sum(mol.nelec),2))

56.00000000000002 28.0
55.99999999999986 28.0
